In [1]:
import numpy as np
import sys
import os
from typing import Sequence
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

sys.path.append(os.path.abspath("../libs"))
sys.path.append(os.path.abspath("../utils"))

from gradient_descendent import gradient_descendent
from normalize import MinMaxNormalizer, StandardScaler
from loss_fn_tarefa2 import make_mse_loss_function, make_mae_loss_function, make_rmse_loss_function
from plots.plots_tarefa2 import plot_best_by_loss

# Ajuste de curva por otimização

## Carregar os dados

In [2]:
# Carregamento dos dados
df = pd.read_excel('../data/Trabalho2dados.xlsx')

df.head()

,x,y,z
0,-5.0,-5.0,-458.963629
1,-5.0,-3.8,-520.361381
2,-5.0,-2.6,-593.039231
3,-5.0,-1.4,-606.776605
4,-5.0,-0.2,-657.401892


## EDA dados

In [3]:
df.describe()

,x,y,z
count,81.000000,81.000000,81.000000
mean,-0.200000,-0.200000,36.990573
std,3.117691,3.117691,294.680404
min,-5.000000,-5.000000,-657.401892
25%,-2.600000,-2.600000,-42.780939
50%,-0.200000,-0.200000,59.403645
75%,2.200000,2.200000,170.599518
max,4.600000,4.600000,614.700100


In [4]:
df.isna().sum()

x    0
y    0
z    0
dtype: int64

In [5]:
df_pivot = df.pivot(index='y', columns='x', values='z')

# Gerar a malha
x_axis = df_pivot.columns.values
y_axis = df_pivot.index.values
z_grid = df_pivot.values

fig = go.Figure()

# Adicionar a Superfície
fig.add_trace(go.Surface(
    x=x_axis,
    y=y_axis,
    z=z_grid,
    colorscale='Viridis',
    opacity=0.9,
    contours_z=dict(show=True, usecolormap=True, project_z=True, highlightcolor="white"),
    name='Superfície dos Dados'
))

# Adicionar os Pontos de Dados Originais
fig.add_trace(go.Scatter3d(
    x=df['x'],
    y=df['y'],
    z=df['z'],
    mode='markers',
    marker=dict(size=3, color='red', symbol='circle'),
    name='Pontos de Dados Originais'
))

# Melhorar o Layout
fig.update_layout(
    title=dict(text='z = f(x, y)', x=0.5),
    scene=dict(
        xaxis_title='Eixo X',
        yaxis_title='Eixo Y',
        zaxis_title='Eixo Z (Valor)'
    ),
    margin=dict(l=0, r=0, b=0, t=50)
)

fig.show()

In [6]:
X = np.column_stack([df['x'] ** 3, df['y'] ** 2, np.ones(len(df['x']))])

true_weights = np.linalg.inv((X.T @ X) + (X.T @ X).T) @ (2 * X.T @ df['z'])
true_weights

array([ 5.0081046 ,  5.73419945, 10.59963756])

## Calcular as funções de perda

### Configurações

In [7]:
def make_function(w):
    def f(x, y):
        if np.isscalar(x) and np.isscalar(y):
            return w @ np.array([x**3, y**2, 1])
        else:
            return np.array([w @ np.array([xi**3, yi**2, 1]) for xi, yi in zip(x, y)])
    return f

features = df[['x', 'y']]
y = df['z']

# Min-Max Normalization
minmax_scaler_x = MinMaxNormalizer(-1, 1)
minmax_scaler_x.fit(features['x'].values)
minmax_scaler_y = MinMaxNormalizer(-1, 1)
minmax_scaler_y.fit(features['y'].values)
minmax_scaler_z = MinMaxNormalizer(-1, 1)
minmax_scaler_z.fit(y.values)

features_normalized = features.copy()
features_normalized['x'] = minmax_scaler_x.normalize(features['x'].values)
features_normalized['y'] = minmax_scaler_y.normalize(features['y'].values)
y_normalized = minmax_scaler_z.normalize(y.values)

# Padronização (Z-score)
std_scaler_x = StandardScaler()
std_scaler_x.fit(features['x'].values)
std_scaler_y = StandardScaler()
std_scaler_y.fit(features['y'].values)
std_scaler_z = StandardScaler()
std_scaler_z.fit(y.values)

features_standardized = features.copy()
features_standardized['x'] = std_scaler_x.normalize(features['x'].values)
features_standardized['y'] = std_scaler_y.normalize(features['y'].values)
y_standardized = std_scaler_z.normalize(y.values)

features_list = [features, features_normalized, features_standardized]
features_names = ['Original', 'Normalized', 'Standardized']
targets_list = [y, y_normalized, y_standardized]

loss_fn_names = ['MSE', 'MAE', 'RMSE']
loss_fn_makers = [make_mse_loss_function, make_mae_loss_function, make_rmse_loss_function]

learning_rates = [0.001, 0.0001, 0.00001]
initial_weights = [
    np.zeros(3),  # Zero sempre seguro
    np.array([0.1, 0.1, 0.1]),  # Valores pequenos
    np.random.randn(3) * 0.01  # Inicialização normal pequena
]
n_iterations = 10000
tolerance = 1e-6

### Rodar os experimentos

In [8]:
dict_results = {}

for feature_set, feature_name, target_data in zip(features_list, features_names, targets_list):
    x_data = feature_set['x'].values
    y_data = feature_set['y'].values
    z_data = target_data

    x_data_orig = features['x'].values
    y_data_orig = features['y'].values
    z_data_orig = y

    for loss_fn_name, loss_fn_maker in zip(loss_fn_names, loss_fn_makers):
        loss_function, grad_loss_function = loss_fn_maker(x_data, y_data, z_data)

        for lr in learning_rates:
            for initial_w in initial_weights:
                try:
                    weights, losses, n_iters = gradient_descendent(
                        initial_w, loss_function, grad_loss_function,
                        learning_rate=lr, max_iter=n_iterations, tolerance=tolerance, stopping_criteria=[1, 2, 3]
                    )

                    # Verificar se há overflow
                    if np.any(np.isnan(weights[-1])) or np.any(np.isinf(weights[-1])):
                        print(f"Overflow detectado para LR={lr}, Initial_W={initial_w}")
                        continue

                except Exception as e:
                    print(f"Erro para LR={lr}, Initial_W={initial_w}: {e}")
                    continue

                if feature_name == 'Standardized':
                    weights[-1] = std_scaler_x.desnormalize_weights(weights[-1])
                elif feature_name == 'Normalized':
                    weights[-1] = minmax_scaler_x.desnormalize_weights(weights[-1])

                function_aprox = make_function(weights[-1])

                mse_final = np.mean((z_data_orig - function_aprox(x_data_orig, y_data_orig)) ** 2)
                rmse_final = np.sqrt(mse_final)
                mae_final = np.mean(np.abs(z_data_orig - function_aprox(x_data_orig, y_data_orig)))

                key = (feature_name, loss_fn_name, lr, tuple(initial_w))
                dict_results[key] = {
                    'weights': weights,
                    'losses': losses,
                    'n_iters': n_iters,
                    'mse_final': mse_final,
                    'rmse_final': rmse_final,
                    'mae_final': mae_final
                }

df_result = pd.DataFrame([
    {
        'Feature_Set': key[0],
        'Loss_Function': key[1],
        'Learning_Rate': key[2],
        'Initial_Weights': key[3],
        'Final_Weights': value['weights'][-1],
        'Final_Loss': value['losses'][-1],
        'MSE_Final': value['mse_final'],
        'RMSE_Final': value['rmse_final'],
        'MAE_Final': value['mae_final'],
        'Iterations': value['n_iters']
    }
    for key, value in dict_results.items()
])

df_result_to_save = df_result.copy()
df_result_to_save['Learning_Rate'] = df_result_to_save['Learning_Rate'].apply(lambda x: f"{float(x):.6g}")
df_result_to_save['Initial_Weights'] = df_result_to_save['Initial_Weights'].apply(lambda x: np.array(x).tolist())
df_result_to_save['Final_Weights'] = df_result_to_save['Final_Weights'].apply(lambda x: np.array(x).tolist())

df_result_to_save['Initial_Weights'] = df_result_to_save['Initial_Weights'].apply(lambda x: str(x))
df_result_to_save['Final_Weights'] = df_result_to_save['Final_Weights'].apply(lambda x: str(x))

df_result_to_save.to_excel('../output/tarefa2_results.xlsx', index=False, float_format="%.6g")

df_result.sort_values(by='MAE_Final').groupby('Loss_Function').head(3).reset_index(drop=True)

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\.venv\Lib\site-packages\numpy\_core\_methods.py:51: RuntimeWarning:

overflow encountered in reduce

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

overflow encountered in square

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\libs\gradient_descendent.py:65: RuntimeWarning:

invalid value encountered in scalar subtract

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\.venv\Lib\site-packages\pandas\core\series.py:3255: RuntimeWarning:

overflow encountered in dot

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\.venv\Lib\site-packages\pandas\core\series.py:3255: RuntimeWarning:

invalid value encountered in dot

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\libs\gradient_descendent.py:51: RuntimeWarning:

invalid value encountered in subtract



Overflow detectado para LR=0.001, Initial_W=[0. 0. 0.]
Overflow detectado para LR=0.001, Initial_W=[0.1 0.1 0.1]
Overflow detectado para LR=0.001, Initial_W=[-0.0047014  -0.01468584 -0.00798965]


,Feature_Set,Loss_Function,Learning_Rate,Initial_Weights,Final_Weights,Final_Loss,MSE_Final,RMSE_Final,MAE_Final,Iterations
0,Original,MSE,0.0001,"(0.1, 0.1, 0.1)","[5.004883393524895, 5.981410684267237, 6.34990...",379.972200,379.972200,19.492876,15.316278,10000
1,Original,MSE,0.0001,"(0.0, 0.0, 0.0)","[5.004853554896708, 5.983700650964379, 6.31053...",380.118869,380.118869,19.496637,15.318501,10000
2,Original,MSE,0.0001,"(-0.004701399300919029, -0.01468584495400809, ...","[5.004851294993272, 5.983874087342691, 6.30755...",380.130033,380.130033,19.496924,15.318669,10000
3,Original,RMSE,0.0010,"(0.1, 0.1, 0.1)","[5.001863826357645, 6.213147485618186, 2.36617...",20.041726,401.670789,20.041726,15.672183,10000
4,Original,RMSE,0.0010,"(0.0, 0.0, 0.0)","[5.001803905449378, 6.217746118086512, 2.28712...",20.055960,402.241520,20.055960,15.683177,10000
5,Original,RMSE,0.0010,"(-0.004701399300919029, -0.01468584495400809, ...","[5.0017993145268, 6.218098448620902, 2.2810692...",20.057055,402.285473,20.057055,15.684019,10000
6,Original,MAE,0.0010,"(0.0, 0.0, 0.0)","[5.01816187654278, 6.283904691357363, 1.135543...",15.825815,412.167477,20.301908,15.825815,4949
7,Original,MAE,0.0010,"(0.1, 0.1, 0.1)","[5.0166004938267905, 6.285862716048745, 1.1116...",15.829857,412.194659,20.302578,15.829857,4256
8,Original,MAE,0.0010,"(-0.004701399300919029, -0.01468584495400809, ...","[5.018030551316137, 6.292021809366623, 0.94740...",15.851847,413.744175,20.340702,15.851847,3629


## Resultados

### MSE

In [9]:
_ = plot_best_by_loss(df_result, "MSE", df, features_normalized, features_standardized, show_original_points=True)

Melhor resultado MSE:
Feature Set: Original
Learning Rate: 0.0001
Final Loss: 379.9722000819796
Pesos: [5.00488339 5.98141068 6.34990042]


In [10]:
df_result_mse = df_result[df_result['Loss_Function'] == 'MSE'].sort_values(by='MSE_Final').reset_index(drop=True)
df_result_mse

,Feature_Set,Loss_Function,Learning_Rate,Initial_Weights,Final_Weights,Final_Loss,MSE_Final,RMSE_Final,MAE_Final,Iterations
0,Original,MSE,0.00010,"(0.1, 0.1, 0.1)","[5.004883393524895, 5.981410684267237, 6.34990...",379.972200,379.972200,19.492876,15.316278,10000
1,Original,MSE,0.00010,"(0.0, 0.0, 0.0)","[5.004853554896708, 5.983700650964379, 6.31053...",380.118869,380.118869,19.496637,15.318501,10000
2,Original,MSE,0.00010,"(-0.004701399300919029, -0.01468584495400809, ...","[5.004851294993272, 5.983874087342691, 6.30755...",380.130033,380.130033,19.496924,15.318669,10000
3,Original,MSE,0.00001,"(0.1, 0.1, 0.1)","[5.001058032080917, 6.274988199168381, 1.30309...",409.802224,409.802224,20.243572,15.820022,10000
4,Original,MSE,0.00001,"(0.0, 0.0, 0.0)","[5.0009927583711935, 6.279997632595718, 1.2169...",410.504096,410.504096,20.260901,15.831998,10000
5,Original,MSE,0.00001,"(-0.004701399300919029, -0.01468584495400809, ...","[5.0009878147028735, 6.280377034615894, 1.2104...",410.557517,410.557517,20.262219,15.832905,10000
6,Normalized,MSE,0.00100,"(0.1, 0.1, 0.1)","[3.930604415727224, 0.7962720760262926, 18.932...",0.004094,7074.886078,84.112342,65.729220,4938
7,Normalized,MSE,0.00100,"(0.0, 0.0, 0.0)","[3.9401186769317844, 0.7441333479667095, 18.76...",0.004173,7093.167959,84.220947,65.833271,5234
8,Normalized,MSE,0.00100,"(-0.004701399300919029, -0.01468584495400809, ...","[3.9425892296563205, 0.7324278011489092, 18.73...",0.004194,7095.232976,84.233206,65.845791,5262
9,Normalized,MSE,0.00010,"(0.1, 0.1, 0.1)","[2.0122715556581316, 0.5249899077255449, 10.20...",0.059555,33154.193763,182.082931,136.877658,10000


### RMSE

In [11]:
_ = plot_best_by_loss(df_result, "RMSE", df, features_normalized, features_standardized, show_original_points=True)

Melhor resultado RMSE:
Feature Set: Original
Learning Rate: 0.001
Final Loss: 20.041726203384506
Pesos: [5.00186383 6.21314749 2.36617984]


In [12]:
df_result_rmse = df_result[df_result['Loss_Function'] == 'RMSE'].sort_values(by='RMSE_Final').reset_index(drop=True)
df_result_rmse

,Feature_Set,Loss_Function,Learning_Rate,Initial_Weights,Final_Weights,Final_Loss,MSE_Final,RMSE_Final,MAE_Final,Iterations
0,Original,RMSE,0.00100,"(0.1, 0.1, 0.1)","[5.001863826357645, 6.213147485618186, 2.36617...",20.041726,401.670789,20.041726,15.672183,10000
1,Original,RMSE,0.00100,"(0.0, 0.0, 0.0)","[5.001803905449378, 6.217746118086512, 2.28712...",20.055960,402.241520,20.055960,15.683177,10000
2,Original,RMSE,0.00100,"(-0.004701399300919029, -0.01468584495400809, ...","[5.0017993145268, 6.218098448620902, 2.2810692...",20.057055,402.285473,20.057055,15.684019,10000
3,Original,RMSE,0.00010,"(0.1, 0.1, 0.1)","[5.000111977584725, 6.291770872525102, 0.60501...",20.390182,415.759530,20.390182,15.922895,10000
4,Original,RMSE,0.00010,"(0.0, 0.0, 0.0)","[5.000004796867698, 6.295228044651212, 0.51059...",20.410840,416.602406,20.410840,15.937325,10000
5,Original,RMSE,0.00010,"(-0.004701399300919029, -0.01468584495400809, ...","[4.999994626145163, 6.295381458559698, 0.50335...",20.412455,416.668304,20.412455,15.938461,10000
6,Normalized,RMSE,0.00100,"(-0.004701399300919029, -0.01468584495400809, ...","[4.191946952006051, 0.9289434045675888, 20.495...",0.055183,5256.822199,72.503946,56.737538,2530
7,Normalized,RMSE,0.00100,"(0.0, 0.0, 0.0)","[4.1910407611026725, 0.9296492128322519, 20.49...",0.055180,5260.520832,72.529448,56.758110,2494
8,Normalized,RMSE,0.00100,"(0.1, 0.1, 0.1)","[4.185953702560411, 0.9345755699492841, 20.493...",0.055161,5279.893304,72.662874,56.865276,2187
9,Original,RMSE,0.00001,"(0.1, 0.1, 0.1)","[4.4955877486300215, 0.7459156620336218, 0.137...",78.332711,6136.013689,78.332711,63.852982,10000


### MAE

In [13]:
_ = plot_best_by_loss(df_result, "MAE", df, features_normalized, features_standardized, show_original_points=True)

Melhor resultado MAE:
Feature Set: Original
Learning Rate: 0.001
Final Loss: 15.825814843820108
Pesos: [5.01816188 6.28390469 1.13554321]


In [14]:
df_result_mae = df_result[df_result['Loss_Function'] == 'MAE'].sort_values(by='MAE_Final').reset_index(drop=True)
df_result_mae

,Feature_Set,Loss_Function,Learning_Rate,Initial_Weights,Final_Weights,Final_Loss,MSE_Final,RMSE_Final,MAE_Final,Iterations
0,Original,MAE,0.00100,"(0.0, 0.0, 0.0)","[5.01816187654278, 6.283904691357363, 1.135543...",15.825815,412.167477,20.301908,15.825815,4949
1,Original,MAE,0.00100,"(0.1, 0.1, 0.1)","[5.0166004938267905, 6.285862716048745, 1.1116...",15.829857,412.194659,20.302578,15.829857,4256
2,Original,MAE,0.00100,"(-0.004701399300919029, -0.01468584495400809, ...","[5.018030551316137, 6.292021809366623, 0.94740...",15.851847,413.744175,20.340702,15.851847,3629
3,Original,MAE,0.00010,"(0.1, 0.1, 0.1)","[5.007548316049207, 6.24645397530912, 0.644371...",15.919502,416.256517,20.402365,15.919502,9749
4,Original,MAE,0.00010,"(0.0, 0.0, 0.0)","[5.00462019753064, 6.232967753086868, 0.549053...",15.945192,417.440771,20.431367,15.945192,9521
5,Original,MAE,0.00010,"(-0.004701399300919029, -0.01468584495400809, ...","[4.999905346377917, 6.207119537762429, 0.53767...",15.976403,418.293963,20.452236,15.976403,9295
6,Normalized,MAE,0.00100,"(0.1, 0.1, 0.1)","[3.9620777777777665, 0.9192296296296283, 19.54...",0.045529,6662.468216,81.623944,63.734031,2695
7,Normalized,MAE,0.00100,"(0.0, 0.0, 0.0)","[3.946503703703762, 0.9145925925925888, 19.465...",0.045643,6776.532042,82.319694,64.275194,2909
8,Normalized,MAE,0.00100,"(-0.004701399300919029, -0.01468584495400809, ...","[3.9458777278000894, 0.9143005368133545, 19.46...",0.045647,6781.313481,82.348731,64.297776,2927
9,Original,MAE,0.00001,"(0.1, 0.1, 0.1)","[3.617496393086131, 0.6403988641975492, 0.1449...",85.443627,11309.454215,106.345918,85.443627,10000
